Testing to see if the membership function used in `Iron_Abell2151-Virgo_ITFR_MJ.ipynb` is causing a problem or not (it isn't).

# DESI Iron
Determine how many galaxies were observed in DESI Iron that are good TF targets by finding the number of unique galaxies observed with observations at the center and at .4R.

In [1]:
## Module imports and constant definitions 
from astropy.table import unique, Table
from astropy.coordinates import SkyCoord
from astropy.io import fits 

import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 

from tqdm.auto import tqdm

import os

import requests

from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

h = 1
H0 = 100*h
c = 3e5
q0 = 0.2
V0 = 2.5 # 0-point of TFR

### Iron Data
We read in the Iron data to use throughout the notebook. Commented out below is the Fuji data, to use to check against when necessary. 

In [2]:
tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_iron_healpix.fits')
#tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits')

tiron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061


### SGA
We read in the SGA to use throughout the notebook. 

In [3]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

## Create a dictionary of SGA IDs to find row indices 
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i

SGA[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

## Separation between Galaxies and Observations
Find all targets on each galaxy, and then calculate distance between SGA center coordinates and observation coordinates. 

In [ ]:
tiron['SKY_FIBER_DIST'] = 0.
tiron['SKY_FIBER_DIST_R26'] = 0.

## For each SGA galaxy that has 1+ observations, calculate distance for all of its targets
for sga_id in tqdm(np.unique(tiron['SGA_ID'])):
    
    ## Identify all galaxy targets on this galaxy
    obs_idx = tiron['SGA_ID'] == sga_id
    
    ## Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    ## Calculate distance between each observation and the center of the galaxy
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tiron['RA'][obs_idx], 
                             dec=tiron['DEC'][obs_idx], 
                             unit=u.degree)
    sep2d = target_coords.separation(SGA_coords)
    
    ## Add the distance to the tiron table
    tiron['SKY_FIBER_DIST'][obs_idx] = sep2d
    ## Add the distance in R26 to the tiron table 
    tiron['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)

## Isolate the centers to be those measurements where the distance is <.1*R26
centers_boolean = tiron['SKY_FIBER_DIST_R26'] < 0.1

## Sort observations into center and axis observations 
iron_centers = tiron[centers_boolean]
iron_axis = tiron[~centers_boolean]

  0%|          | 0/75152 [00:00<?, ?it/s]

## Cleaning Iron Center Observations

Only keep those observations with
 * `DELTACHI2` > 25
 * `ZWARN` == 0

In [ ]:
good_centers = iron_centers[(iron_centers['DELTACHI2'] > 25) & (iron_centers['ZWARN'] == 0)]

## Check for multiple good center observations 
unique_ids, counts = np.unique(good_centers['SGA_ID'], return_counts=True)

If there's at least one good center observation, set the galaxy's redshift.

In [ ]:
SGA['Z_DESI'] = np.nan
SGA['ZERR_DESI'] = np.nan

weights = 1./(good_centers['ZERR']**2)

for sga_id in np.unique(good_centers['SGA_ID']):
    
    ## Find all the center observations of this galaxy
    obs_idx = good_centers['SGA_ID'] == sga_id
    
    ## Find the row in SGA for this galaxy
    SGA_idx = SGA_dict[sga_id]
    
    # Set the redshift of this galaxy to be weighted average of all good center observation redshifts
    SGA['Z_DESI'][SGA_idx] = np.average(good_centers['Z'][obs_idx], 
                                        weights=weights[obs_idx])
    SGA['ZERR_DESI'][SGA_idx] = np.sqrt(1./np.sum(weights[obs_idx]))

Determine how many observations at $.4R$ we have. 

In [ ]:
## Include all observations where .38R < distance < .42R in our .4R values
r0p4 = iron_axis[(iron_axis['SKY_FIBER_DIST_R26'] > 0.38) & (iron_axis['SKY_FIBER_DIST_R26'] < 0.42)]

## Count how many of these are unique values
unique_centers = np.unique(good_centers['SGA_ID'])
unique_r0p4 = np.unique(r0p4['SGA_ID'])

centers_and_p4s = []
for i in unique_r0p4: 
    if i in unique_centers: 
        centers_and_p4s.append(i)

## Output the number of observations that are generally good for TF fitting
print(len(centers_and_p4s)," unique galaxies with center and .4R observations")

## Find the percentage of all Iron galaxies that are generally good for TF fitting
num_iron = len(np.unique(tiron['SGA_ID']))
percentage_TF = 100*((len(centers_and_p4s))/num_iron)
print(percentage_TF,"% of all Iron galaxies")

## Cluster Membership
Following Cosmicflows4 (Kourkchi et al. 2020), cluster membership is defined as
- $R_p < 1.5R_{2t}$ and $v < V_c \pm 3\sigma_p$
- $1.5R_{2t} \leq R_p < 3R_{2t}$ and $v < V_c \pm 2\sigma_p$

where $R_p$ is the projected distance from the cluster center, $R_{2t}$ is the cluster projected second turnaround radius, $\sigma_p$ is the projected velocity dispersion of the cluster, and $V_c$ is the average heliocentric radial velocity of the cluster.

### Tully et. al. (2015) Table 3

In [ ]:
hdu = fits.open("../Tully15-Table3.fits")
table3 = Table(hdu[1].data)
table3.rename_column("<Vcmba>","VMod")
hdu.close()

table3_dict = {}
for i in range(len(table3)):
    table3_dict[table3['Nest'][i]] = i

table3[:5]

In [ ]:
## Check a suspicious value from later on in the notebook
table3[table3['Nest'] == 100526]

In [ ]:
## Continue the suspicious value check 
cluster_coord = SkyCoord(table3['SGLON'][table3['Nest'] == 100526]*u.degree, 
                         table3['SGLAT'][table3['Nest'] == 100526]*u.degree, 
                         frame='supergalactic')
print(cluster_coord.transform_to('icrs'))

### Tully et. al (2013) Table 2

In [ ]:
hdu = fits.open("../Tully13-Table2.fit")
table2 = Table(hdu[1].data)
hdu.close()

table2[:5]

Loop through the SGA table to do just the rows that are for IDs we want from centers_and_p4s. Collect them into a smaller table. 

In [ ]:
SGA.add_index("SGA_ID")
TF_SGA_cp4 = SGA.loc[centers_and_p4s]

TF_SGA_cp4[:5]

In [ ]:
## Cluster Membership function. Takes argument of nest ID for the target cluster.

"""def cluster_membership(nest_id):
    c_4r_in_cluster = []
    
    ## Set active row for each nest number and base values 
    active_row = table3_dict[nest_id]
    R2t = table3["R2t"][active_row]
    sigma = table3["sigP"][active_row]
    
    ## Find the coordinates for each cluster
    cluster_coords = SkyCoord(table3["SGLON"][active_row]*u.degree, 
                       table3["SGLAT"][active_row]*u.degree, 
                       frame='supergalactic')
    
    group_coords = SkyCoord(table2['SGLON']*u.degree, 
                        table2['SGLAT']*u.degree, 
                        frame='supergalactic')
    
    ## Match cluster to group coordinates 
    idx, d2d, d3d = cluster_coords.match_to_catalog_sky(group_coords)
    v = table2["__HV_"][idx]
    
    ## Match SGA coordinates from unique galaxies to nearest cluster
    SGA_coords = SkyCoord(TF_SGA_cp4['RA'], TF_SGA_cp4['DEC'], unit='deg')
    sep = cluster_coords.separation(SGA_coords)

    ## Convert R2t to an angle 
    R2t_angle = (R2t/(v/H0))*u.radian
    
    SGA_in_cluster1 = (sep < 1.5*R2t_angle) & (TF_SGA_cp4["Z_DESI"]*c > v - 3*sigma) & (TF_SGA_cp4["Z_DESI"]*c < v + 3*sigma)
    SGA_in_cluster2 = (sep >= 1.5*R2t_angle) & (sep < 3*R2t_angle) & (TF_SGA_cp4["Z_DESI"]*c > v - 2*sigma) & (TF_SGA_cp4["Z_DESI"]*c < v + 2*sigma)
    
    SGA_in_cluster = SGA_in_cluster1 | SGA_in_cluster2
    
    ## Keep observations that are within nest cluster 
    SGA_ID_in_cluster = TF_SGA_cp4["SGA_ID"][SGA_in_cluster]
    
    ## Gather centers and axes in the cluster
    centers_in_cluster = good_centers[np.in1d(good_centers['SGA_ID'], SGA_ID_in_cluster)]
    axis_in_cluster = iron_axis[np.in1d(iron_axis['SGA_ID'], SGA_ID_in_cluster)]
    
    c_4r_in_cluster.append(SGA_ID_in_cluster)
    return(c_4r_in_cluster, SGA_in_cluster, SGA_ID_in_cluster, centers_in_cluster, axis_in_cluster, cluster_coords, v, sep)"""

## Virgo Explicit Filtering
Testing just Virgo with the membership and cutting criteria.

In [ ]:
c_4r_in_Virgo = []
    
## Set active row for Virgo nest and base values 
active_row_Virgo = table3_dict[100002]
R2t_Virgo = table3["R2t"][active_row_Virgo]
sigma_Virgo = table3["sigP"][active_row_Virgo]

## Find the coordinates for each cluster
Virgo_coords = SkyCoord(table3["SGLON"][active_row_Virgo]*u.degree, 
                   table3["SGLAT"][active_row_Virgo]*u.degree, 
                   frame='supergalactic')

group_coords = SkyCoord(table2['SGLON']*u.degree, 
                    table2['SGLAT']*u.degree, 
                    frame='supergalactic')

## Match cluster to group coordinates 
idx_Virgo, d2d_Virgo, d3d_Virgo = Virgo_coords.match_to_catalog_sky(group_coords)
v_Virgo = table2["__HV_"][idx_Virgo]

## Match SGA coordinates from unique galaxies to nearest cluster
SGA_coords_Virgo = SkyCoord(TF_SGA_cp4['RA'], TF_SGA_cp4['DEC'], unit='deg')
sep_Virgo = Virgo_coords.separation(SGA_coords_Virgo)

## Convert R2t to an angle 
R2t_Virgo_angle = (R2t_Virgo/(v_Virgo/H0))*u.radian

SGA_in_cluster1_Virgo = (sep_Virgo < 1.5*R2t_Virgo_angle) & (TF_SGA_cp4["Z_DESI"]*c > v_Virgo - 3*sigma_Virgo) & (TF_SGA_cp4["Z_DESI"]*c < v_Virgo + 3*sigma_Virgo)
SGA_in_cluster2_Virgo = (sep_Virgo >= 1.5*R2t_Virgo_angle) & (sep_Virgo < 3*R2t_Virgo_angle) & (TF_SGA_cp4["Z_DESI"]*c > v_Virgo - 2*sigma_Virgo) & (TF_SGA_cp4["Z_DESI"]*c < v_Virgo + 2*sigma_Virgo)

SGA_in_Virgo = SGA_in_cluster1_Virgo | SGA_in_cluster2_Virgo

## Keep observations that are within Virgo-2151
SGA_ID_in_Virgo = TF_SGA_cp4["SGA_ID"][SGA_in_Virgo]

## Gather centers and axes in the cluster
centers_in_Virgo = good_centers[np.in1d(good_centers['SGA_ID'], SGA_ID_in_Virgo)]
axis_in_Virgo = iron_axis[np.in1d(iron_axis['SGA_ID'], SGA_ID_in_Virgo)]

c_4r_in_Virgo.append(SGA_ID_in_Virgo)

In [ ]:
## make a bar graph of the angular separation 
plt.hist(sep_Virgo[SGA_in_Virgo].to_value('degree'), bins=np.arange(0,25, 1))
plt.xlabel('SGA-Virgo Angular Separation [deg]')
plt.ylabel('number of galaxies');

In [ ]:
## plot the physical locations and the redshifts of the cluster

plt.figure(figsize=(15,5), tight_layout=True)

plt.subplot(131)
plt.plot(centers_in_Virgo['TARGET_RA'], centers_in_Virgo['TARGET_DEC'], '.')
plt.plot(Virgo_coords.transform_to('icrs').ra.deg, Virgo_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel(r'$\alpha$ [deg]')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(132)
plt.plot(centers_in_Virgo['Z'], centers_in_Virgo['TARGET_DEC'], '.')
plt.plot(v_Virgo/c, Virgo_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel('z')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(133)
plt.hist(centers_in_Virgo['Z'], bins=np.arange(0.0, .01, 0.001))
plt.vlines(v_Virgo/c, 0, 100, colors='k', linestyles='dotted')
plt.xlabel('redshift')
plt.ylabel('number of galaxies')
plt.ylim(ymax=95);

In [ ]:
axis_SGAids_Virgo, axis_counts_Virgo = np.unique(axis_in_Virgo['SGA_ID'], return_counts=True)
center_SGAids_Virgo, center_counts_Virgo = np.unique(centers_in_Virgo['SGA_ID'], return_counts=True)

counts_Virgo = []

for sga_id in SGA_ID_in_Virgo:
    
    center_count_Virgo = 0
    axis_count_Virgo = 0
    
    if sga_id in center_SGAids_Virgo:
        
        center_count_Virgo = center_counts_Virgo[center_SGAids_Virgo == sga_id]
        
    if sga_id in axis_SGAids_Virgo:
        
        axis_count_Virgo = axis_counts_Virgo[axis_SGAids_Virgo == sga_id]
        
    count = center_count_Virgo + axis_count_Virgo
    
    if count > 1:
        
        counts_Virgo.append(count)
        

plt.figure(tight_layout=True)

plt.hist(np.array(counts), bins=np.arange(2,15))

plt.xlabel('Observations per SGA_ID in Virgo-2151')
plt.ylabel('count');

### Calculate the rotational velocity

In [ ]:
axis_in_Virgo['SKY_FIBER_DIST'] = 0.
axis_in_Virgo['SKY_FIBER_DIST_R26'] = 0.
axis_in_Virgo['V_ROT'] = np.nan
axis_in_Virgo['V_ROT_ERR'] = np.nan


# For each SGA galaxy that has at least one center observation, calculate the 
# distance for all of that galaxy's targets
for sga_gal in np.unique(centers_in_Virgo['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = axis_in_Virgo['SGA_ID'] == sga_gal
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_gal]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center
    #---------------------------------------------------------------------------
    center_coords_Virgo = SkyCoord(ra=SGA['RA'][sga_idx], 
                             dec=SGA['DEC'][sga_idx], 
                             unit=u.degree)
    target_coords_Virgo = SkyCoord(ra=axis_in_Virgo['RA'][obs_idx], 
                             dec=axis_in_Virgo['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d_Virgo = target_coords_Virgo.separation(center_coords_Virgo)
    
    axis_in_Virgo['SKY_FIBER_DIST'][obs_idx] = sep2d_Virgo
    axis_in_Virgo['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d_Virgo.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Calculate rotational velocity
    #---------------------------------------------------------------------------
    # Use the average redshift of all center observations for the systemic velocity
    z_center_Virgo = np.mean(SGA['Z_DESI'][sga_idx])
    z_err_center2_Virgo = SGA['ZERR_DESI'][sga_idx]**2

    # Calculate rotational velocity for all observations of the galaxy
    axis_in_Virgo['V_ROT'][obs_idx] = c*(axis_in_Virgo['Z'][obs_idx] - z_center_Virgo)
    axis_in_Virgo['V_ROT_ERR'][obs_idx] = c*np.sqrt(axis_in_Virgo['ZERR'][obs_idx]**2 + z_err_center2_Virgo)
    #---------------------------------------------------------------------------
    
    
    #---------------------------------------------------------------------------
    # Correct rotational velocities for inclination angle
    #---------------------------------------------------------------------------
    cosi2 = (SGA['BA'][sga_idx]**2 - q0**2)/(1 - q0**2)
    
    # Galaxies with b/a < q0
    if cosi2 < 0:
        cosi2 = 0
    
    axis_in_Virgo['V_ROT'][obs_idx] /= np.sin(np.arccos(np.sqrt(cosi2)))
    #---------------------------------------------------------------------------

In [ ]:
plt.figure(tight_layout=True)

plt.hist(np.abs(axis_in_Virgo['V_ROT']), bins=np.linspace(0, 1000, 100))

plt.xlabel('$V_{rot}$ [km/s]')
plt.ylabel('number of observations');

### Cut for Virgo galaxies suitable for calibrating the TFR

Requirements:
 * $10 < V_{rot} < 1000$ km/s at $0.33R_{26}$
 * $\Delta V / V_{min} \leq 5$
 * $i > 45^\circ$
 * Spiral-type morphology
 * Passes visual inspection

#### Velocity Cut

In [ ]:
r0p4_Virgo = (axis_in_Virgo['SKY_FIBER_DIST_R26'] > 0.38) & (axis_in_Virgo['SKY_FIBER_DIST_R26'] < 0.42)

Vgood_Virgo = (np.abs(axis_in_Virgo['V_ROT']) < 1000) & (np.abs(axis_in_Virgo['V_ROT']) > 10)

good_axis_in_Virgo = axis_in_Virgo[r0p4_Virgo & Vgood_Virgo]

print(len(good_axis_in_Virgo), len(np.unique(good_axis_in_Virgo['SGA_ID'])))

#### Relative Velocity Cut

In [ ]:
good_deltaV_Virgo = np.ones(len(good_axis_in_Virgo), dtype=bool)

for sga_id in np.unique(good_axis_in_Virgo['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = good_axis_in_Virgo['SGA_ID'] == sga_id
    
    n_obs = np.sum(obs_idx)
    
    if n_obs > 1:
        
        Vmin_Virgo = np.min(np.abs(good_axis_in_Virgo['V_ROT'][obs_idx]))
        Vmax_Virgo = np.max(np.abs(good_axis_in_Virgo['V_ROT'][obs_idx]))
        
        v_norm_min_Virgo = np.abs(good_axis_in_Virgo['V_ROT'][obs_idx])/Vmin_Virgo
        v_norm_max_Virgo = np.abs(good_axis_in_Virgo['V_ROT'][obs_idx])/Vmax_Virgo
        
        diff_matrix = np.abs(good_axis_in_Virgo['V_ROT'][obs_idx]).reshape(n_obs, 1) - np.abs(good_axis_in_Virgo['V_ROT'][obs_idx]).reshape(1, n_obs)
        
        diff_matrix_norm = diff_matrix/Vmin_Virgo
        
        if np.any(np.abs(diff_matrix_norm) > 5.):
            
            '''
            print(sga_id)
            print(diff_matrix_norm)
            print(1/v_norm_min.data)
            print(v_norm_max.data)
            print(good_axis_inComa[['TARGETID', 'V_ROT', 'PVTYPE', 'FILENAME', 'DELTACHI2', 'ZWARN']][obs_idx].pprint(max_width=-1))
            ''';
            
            # Remove all observations with DELTACHI2 < 25
            # Note: This also typically removes observations with ZWARN != 0
            deltachi2_idx = good_axis_in_Virgo['DELTACHI2'] >= 25
            
            good_deltaV_Virgo[obs_idx & ~deltachi2_idx] = False
            
            good_obs_idx = obs_idx & deltachi2_idx
            
            n_obs_good = np.sum(good_obs_idx)
            
            # Check to make sure that, if there are still multiple observations, they all satisfy our relative velocity criteria
            if n_obs_good > 1:
                
                Vmin_Virgo = np.min(np.abs(good_axis_in_Virgo['V_ROT'][good_obs_idx]))
                
                diff_matrix = np.abs(good_axis_in_Virgo['V_ROT'][good_obs_idx]).reshape(n_obs_good, 1) - np.abs(good_axis_in_Virgo['V_ROT'][good_obs_idx]).reshape(1, n_obs_good)
                
                diff_matrix_norm = diff_matrix/Vmin_Virgo
                
                if np.any(np.abs(diff_matrix_norm) > 5.):
                    '''
                    print(sga_id)
                    print(diff_matrix_norm)
                    print(good_axis_inComa[['TARGETID', 'V_ROT', 'PVTYPE', 'FILENAME', 'DELTACHI2', 'ZWARN']][obs_idx].pprint(max_width=-1))
                    ''';
                    # Set all of these so that we don't look at this galaxy
                    good_deltaV_Virgo[good_obs_idx] = False

In [ ]:
good_deltaV_axis_in_Virgo = good_axis_in_Virgo[good_deltaV_Virgo]

print(len(good_deltaV_axis_in_Virgo), len(np.unique(good_deltaV_axis_in_Virgo['SGA_ID'])))

#### Inclination Angle Cut

In [ ]:
SGA['cosi2'] = (SGA['BA']**2 - q0**2)/(1 - q0**2)
SGA['cosi2'][SGA['cosi2'] < 0] = 0

good_deltaV_axis_in_Virgo['iSGA'] = -1

for i in range(len(good_deltaV_axis_in_Virgo)):
    
    # Find galaxy in SGA
    sga_idx = SGA_dict[good_deltaV_axis_in_Virgo['SGA_ID'][i]]
    
    good_deltaV_axis_in_Virgo['iSGA'][i] = sga_idx
    
good_deltaV_axis_in_Virgo['cosi2'] = SGA['cosi2'][good_deltaV_axis_in_Virgo['iSGA']]

In [ ]:
i_min = 45. # degrees

cosi2_max = np.cos(i_min*np.pi/180.)**2

edge = good_deltaV_axis_in_Virgo['cosi2'] <= cosi2_max

good_edge_axis_in_Virgo = good_deltaV_axis_in_Virgo[edge]

print(len(good_edge_axis_in_Virgo), len(np.unique(good_edge_axis_in_Virgo['SGA_ID'])))

In [ ]:
plt.figure(tight_layout=True)

plt.hist(np.arccos(np.sqrt(good_edge_axis_in_Virgo['cosi2']))*180/np.pi, bins=np.linspace(0, 90, 10))

plt.xlabel('inclination angle [deg]')
plt.ylabel('number of observations');

#### Morphology Cut

In [ ]:
good_edge_axis_in_Virgo['MORPHTYPE'] = SGA['MORPHTYPE'][good_edge_axis_in_Virgo['iSGA']]

In [ ]:
spirals_Virgo = np.zeros(len(good_edge_axis_in_Virgo), dtype=bool)

for i in range(len(good_edge_axis_in_Virgo)):
    
    try:    
        if (good_edge_axis_in_Virgo['MORPHTYPE'][i][0] == 'S') and (good_edge_axis_in_Virgo['MORPHTYPE'][i][:2] != 'S0'):
            spirals_Virgo[i] = True
    except IndexError:
        print(good_edge_axis_in_Virgo['MORPHTYPE'][i])

good_edge_spirals_axis_in_Virgo = good_edge_axis_in_Virgo[spirals_Virgo]

print(len(good_edge_spirals_axis_in_Virgo), len(np.unique(good_edge_spirals_axis_in_Virgo['SGA_ID'])))

In [ ]:
## This needs to be the variable-renamed version of the membership function

In [ ]:
## make a bar graph of the angular separation 
plt.hist(sep_Virgo[SGA_in_Virgo].to_value('degree'), bins=np.arange(5,25, 1))
plt.xlabel('SGA-Virgo Angular Separation [deg]')
plt.ylabel('number of galaxies');

In [ ]:
## plot the physical locations and the redshifts of the cluster

plt.figure(figsize=(15,5), tight_layout=True)

plt.subplot(131)
plt.plot(centers_in_Virgo['TARGET_RA'], centers_in_Virgo['TARGET_DEC'], '.')
plt.plot(Virgo_coords.transform_to('icrs').ra.deg, Virgo_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel(r'$\alpha$ [deg]')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(132)
plt.plot(centers_in_Virgo['Z'], centers_in_Virgo['TARGET_DEC'], '.')
plt.plot(v_Virgo/c, Virgo_coords.transform_to('icrs').dec.deg, 'kx', ms=10, mew=5)
plt.xlabel('z')
plt.ylabel(r'$\delta$ [deg]')

plt.subplot(133)
plt.hist(centers_in_Virgo['Z'], bins=np.arange(0, .01, 0.001))
plt.vlines(v_Virgo/c, 0, 100, colors='k', linestyles='dotted')
plt.xlabel('redshift')
plt.ylabel('number of galaxies')
plt.ylim(ymax=95);

In [ ]:
axis_SGAids, axis_counts = np.unique(axis_in_Virgo['SGA_ID'], return_counts=True)
center_SGAids, center_counts = np.unique(centers_in_Virgo['SGA_ID'], return_counts=True)

counts = []

for sga_id in SGA_ID_in_Virgo:
    
    center_count = 0
    axis_count = 0
    
    if sga_id in center_SGAids:
        
        center_count = center_counts[center_SGAids == sga_id]
        
    if sga_id in axis_SGAids:
        
        axis_count = axis_counts[axis_SGAids == sga_id]
        
    count = center_count + axis_count
    
    if count > 1:
        
        counts.append(count)
        

plt.figure(tight_layout=True)

plt.hist(np.array(counts), bins=np.arange(2,15))

plt.xlabel('Observations per SGA_ID in Virgo')
plt.ylabel('count');